In [1]:
from pathlib import Path
import pandas as pd
from matplotlib.figure import Figure
import xlwings as xw
# Requires a license key: https://www.xlwings.org/trial
from xlwings.pro import Markdown, MarkdownStyle
from xlwings.pro.reports import create_report, Image

### Some data

In [2]:
# Matplotlib
fig = Figure(figsize=(4, 3))
ax = fig.add_subplot(111)
ax.plot([1, 2, 3, 4, 5])

# Pandas DataFrame
perf_data = pd.DataFrame(index=['South', 'North', 'West'],
                         columns=[2020, 2021],
                         data=[[1., 2.], [3., 4.], [5, 6]])
perf_data.index.name = 'Sales'

# Picture
logo = Image(Path('xlwings.jpg').resolve())

# Float
perf = 0.12

In [3]:
# Markdown

mytext = """\
# Q1 2021 Results

The perfomance was {{ perf }}.
This was due to the following points:

* More sales
* Cost cuts

# Sales were strong

*Automation was the most important driver*.
More info on request.
"""

style = MarkdownStyle()
style.h1.font.color = (21, 164, 58)
style.h1.font.size = 14

### Using an invisible App

In [5]:
# Optional: collect data into a dictionary
data = dict(perf_data=perf_data.reset_index(), logo=logo,
            perf=perf, fig=fig, summary=Markdown(mytext, style))

with xw.App(visible=False) as app:
    wb = app.create_report('report_template.xlsx',
                           'report.xlsx',
                            **data)
    wb.to_pdf()

### Render a template on a sheet

In [ ]:
wb = xw.Book('report_template.xlsx')
template_sheet = wb.sheets[0]

In [ ]:
# Copy the template sheet first!
# You could provide a name via copy(name=...)
report_sheet = template_sheet.copy()

In [ ]:
report_sheet.render_template(**data)

In [ ]:
report_sheet.to_pdf()